In [1]:
import os, sys
import numpy as np

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from packages.tcgahandler import LayerDataset

from IPython.display import display
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'

DATA_DIR = "../../data/"

# TCGA-LGG

In [2]:
project = "TCGA-LGG"
layer = "protein"
dataset = LayerDataset(DATA_DIR, project, layer)

In [3]:
col_types = dataset.get_types_of_columns()
head = dataset.get_layer(n_rows=5)
ids = dataset.get_layer_by_column_type("")

print(f"Number of proteins: {ids.shape[0]}\nNumber of patients: {col_types.most_common()[0][1]}")
print(f"Types of columns: {col_types}")
display(head)

Number of proteins: 487
Number of patients: 435
Types of columns: Counter({'': 435, 'AGID': 1, 'lab_id': 1, 'catalog_number': 1, 'set_id': 1, 'peptide_target': 1})


,lab_id,catalog_number,set_id,peptide_target,TCGA-DB-5279-01A,TCGA-HT-7855-01A,TCGA-QH-A6CW-01A,TCGA-DB-A64Q-01A,TCGA-HT-7604-01A,TCGA-FG-7637-01A,...,TCGA-HT-7880-01A,TCGA-QH-A6CX-01A,TCGA-F6-A8O4-01A,TCGA-HT-7470-01A,TCGA-TQ-A7RG-01A,TCGA-P5-A731-01A,TCGA-QH-A65V-01A,TCGA-P5-A5F1-01A,TCGA-S9-A6TY-01A,TCGA-R8-A6YH-01A
AGID,,,,,,,,,,,,,,,,,,,,,
AGID00100,882,sc-628,Old,1433BETA,0.063553,0.097345,0.325500,-0.095779,-0.041091,0.27307,...,0.324480,0.092804,0.62854,0.19147,0.055679,0.50066,0.485570,0.30110,0.23886,0.23749
AGID00111,913,sc-23957,Old,1433EPSILON,0.284310,0.733370,0.095182,0.114320,0.129820,0.34437,...,0.259070,0.077194,0.22880,0.26427,0.064463,0.28235,0.214310,0.46626,0.44848,0.10152
AGID00101,883,sc-1019,Old,1433ZETA,-0.408220,-0.545870,0.042580,-0.340790,-0.131500,-0.22034,...,-0.020128,-0.402860,-0.12014,0.19510,-0.439600,-0.29249,-0.302120,0.26825,0.54965,-0.41721
AGID00001,2,9452,Old,4EBP1,-0.686490,-0.461930,-0.476620,-0.537770,0.024208,-0.24296,...,-0.945280,-0.727870,-0.36614,-0.84887,-0.672660,-0.62856,-0.633870,-0.12012,-0.30425,-0.52234
AGID00002,3,9456,Old,4EBP1_pS65,-0.254240,-0.280650,-0.191060,0.014253,-0.135920,0.17197,...,0.162860,-0.386700,-0.49195,0.34179,-0.185180,-0.12082,-0.011695,-0.18069,0.09742,-0.25235


Check for the presence of NaNs

In [4]:
t = dataset.get_layer_by_column_type('')
t = t.drop(columns=["lab_id", "catalog_number","set_id", "peptide_target"])
print(f"Are there any NaNs?\n{t.isna().any().any()}")

Are there any NaNs?
True


Sparsity analysis - identify and remove columns with high percentage (>95) of 0s

In [5]:
print("fraction of zeros in whole dataset: ", (t == 0).sum().sum() / (t.shape[0]*t.shape[1]))
zero_count = ((t == 0).sum(axis="columns") / t.shape[1])
print(zero_count.sort_values(ascending=False).head(5))

fraction of zeros in whole dataset:  0.0
AGID
AGID00100    0.0
AGID02155    0.0
AGID00245    0.0
AGID02199    0.0
AGID00323    0.0
dtype: float64


Process and save datasets

In [6]:
col_types = dataset.get_types_of_columns()
print("Types of columns:\n", list(col_types.keys()))

Types of columns:
 ['AGID', 'lab_id', 'catalog_number', 'set_id', 'peptide_target', '']


Relative Expression Dataset

In [7]:
type_of_column = ""

df = dataset.get_layer_by_column_type(type_of_column)
df = df.drop(columns=["lab_id", "catalog_number","set_id", "peptide_target"])
print(df.shape)

# drop rows with NaN
df = df.dropna(axis="index", how="any")
print(df.shape)

# transpose dataframe
df = df.T

display(df.head())
dataset.set_raw_data(data_type="rppa", df=df)

(487, 435)
(456, 435)


AGID,AGID00100,AGID00111,AGID00101,AGID00001,AGID00002,AGID00003,AGID00443,AGID00120,AGID00004,AGID00005,...,AGID00349,AGID02137,AGID00088,AGID00089,AGID00504,AGID00095,AGID02217,AGID02210,AGID00326,AGID00432
TCGA-DB-5279-01A,0.063553,0.284310,-0.40822,-0.686490,-0.254240,0.44096,-0.40288,-0.44502,-0.010774,0.432730,...,-0.525998,-0.44341,-0.59079,-2.47700,-0.15133,0.073364,-0.841587,-0.883834,-1.475134,0.658320
TCGA-HT-7855-01A,0.097345,0.733370,-0.54587,-0.461930,-0.280650,0.83087,-0.34850,-0.57879,-0.152990,-0.730700,...,-0.429580,-0.21012,-0.20853,-0.83595,0.23712,0.016250,-0.167003,-0.186865,-1.418856,0.431770
TCGA-QH-A6CW-01A,0.325500,0.095182,0.04258,-0.476620,-0.191060,0.77002,-0.17413,-0.29480,-0.191830,-0.528050,...,0.022217,-0.33641,-0.27488,-0.76129,-0.53427,0.108930,-0.346167,-0.613352,-1.517299,1.191739
TCGA-DB-A64Q-01A,-0.095779,0.114320,-0.34079,-0.537770,0.014253,0.93708,-0.25359,0.12298,0.013886,-0.029776,...,0.306443,-0.37160,-0.37835,-1.14670,-0.56650,0.057059,-0.724308,-0.457130,-1.451234,0.738193
TCGA-HT-7604-01A,-0.041091,0.129820,-0.13150,0.024208,-0.135920,1.26210,-0.35806,0.56085,0.581710,0.527780,...,-0.247401,-0.15366,-0.41184,-0.97738,-0.31331,0.180630,0.062216,-0.050716,-1.581383,0.880428


# TCGA-COAD

In [8]:
project = "TCGA-COAD"
layer = "protein"
dataset = LayerDataset(DATA_DIR, project, layer)

In [9]:
col_types = dataset.get_types_of_columns()
head = dataset.get_layer(n_rows=5)
ids = dataset.get_layer_by_column_type("")

print(f"Number of proteins: {ids.shape[0]}\nNumber of patients: {col_types.most_common()[0][1]}")
print(f"Types of columns: {col_types}")
display(head)

Number of proteins: 727
Number of patients: 363
Types of columns: Counter({'': 363, 'AGID': 1, 'lab_id': 1, 'catalog_number': 1, 'set_id': 1, 'peptide_target': 1})


,lab_id,catalog_number,set_id,peptide_target,TCGA-A6-6141-01A,TCGA-CM-6678-01A,TCGA-D5-6922-01A,TCGA-CM-5341-01A,TCGA-DM-A0X9-01A,TCGA-AA-3502-01A,...,TCGA-AA-3664-01A,TCGA-4T-AA8H-01A,TCGA-DM-A1DB-01A,TCGA-AA-A00J-01A,TCGA-AD-6548-01A,TCGA-G4-6588-01A,TCGA-SS-A7HO-01A,TCGA-A6-6649-01A,TCGA-5M-AATA-01A,TCGA-5M-AAT5-01A
AGID,,,,,,,,,,,,,,,,,,,,,
AGID00100,882,sc-628,Old,1433BETA,-0.250230,0.077985,0.197100,0.10278,-0.12771,-0.073956,...,-0.260810,-0.132220,-0.194620,0.13954,0.142130,0.25254,0.077230,-0.12765,0.002181,0.094542
AGID00111,913,sc-23957,Old,1433EPSILON,-0.171420,0.339400,-0.119910,-0.28875,-0.14024,-0.248680,...,0.059625,0.058415,-0.153290,-0.18585,-0.028797,0.12731,-0.013323,-0.13385,-0.150040,-0.236040
AGID00101,883,sc-1019,Old,1433ZETA,-0.094394,0.694630,0.061087,0.98698,0.10853,0.073882,...,0.501330,0.559630,-0.157790,-0.27101,0.459270,0.62917,-0.084323,0.31820,0.510820,0.053267
AGID00001,2,9452,Old,4EBP1,-0.239470,-0.092303,-0.476420,0.22850,0.73142,0.271650,...,-0.077359,0.788240,-0.059245,-0.20477,0.124960,-0.58954,0.871590,0.15541,0.684380,0.169600
AGID00002,3,9456,Old,4EBP1_pS65,-0.342130,-0.323310,0.614760,0.06715,-0.35289,-0.012491,...,-0.221260,-0.268870,-0.191960,0.49149,0.159070,-0.36609,-0.677020,1.01130,-0.507670,-0.186220


Check for the presence of NaNs

In [10]:
t = dataset.get_layer_by_column_type('')
t = t.drop(columns=["lab_id", "catalog_number","set_id", "peptide_target"])
print(f"Are there any NaNs?\n{t.isna().any().any()}")

Are there any NaNs?
True


Sparsity analysis - identify and remove columns with high percentage (>95) of 0s

In [11]:
print("fraction of zeros in whole dataset: ", (t == 0).sum().sum() / (t.shape[0]*t.shape[1]))
zero_count = ((t == 0).sum(axis="columns") / t.shape[1])
print(zero_count.sort_values(ascending=False).head(5))

fraction of zeros in whole dataset:  0.0
AGID
AGID00100    0.0
AGID02137    0.0
AGID00089    0.0
AGID00504    0.0
AGID00095    0.0
dtype: float64


Process and save datasets

In [12]:
col_types = dataset.get_types_of_columns()
print("Types of columns:\n", list(col_types.keys()))

Types of columns:
 ['AGID', 'lab_id', 'catalog_number', 'set_id', 'peptide_target', '']


Relative Expression Dataset

In [13]:
type_of_column = ""

df = dataset.get_layer_by_column_type(type_of_column)
df = df.drop(columns=["lab_id", "catalog_number","set_id", "peptide_target"])
print(df.shape)

# drop rows with NaN
df = df.dropna(axis="index", thresh=0.8*df.shape[1])
df = df.dropna(axis="columns", thresh=0.5*df.shape[0])
df = df.dropna(axis="index", how="any")
print(df.shape)

# transpose dataframe
df = df.T

display(df.head())
dataset.set_raw_data(data_type="rppa", df=df)

(727, 363)
(455, 351)


AGID,AGID00100,AGID00111,AGID00101,AGID00001,AGID00002,AGID00003,AGID00443,AGID00120,AGID00004,AGID00005,...,AGID00349,AGID02137,AGID00088,AGID00089,AGID00504,AGID00095,AGID02217,AGID02210,AGID00326,AGID00432
TCGA-A6-6141-01A,-0.250230,-0.17142,-0.094394,-0.239470,-0.34213,-0.22751,-0.12147,0.010664,0.70318,0.414530,...,-0.051767,-0.413720,-0.319240,-0.427260,0.17550,0.022338,0.111084,-0.283443,0.126924,0.288428
TCGA-CM-6678-01A,0.077985,0.33940,0.694630,-0.092303,-0.32331,-0.16053,-0.35774,-1.081700,0.19673,0.147650,...,-1.072271,0.129780,0.573590,0.547240,0.36620,-0.063297,0.305689,0.110589,-1.656316,0.088431
TCGA-D5-6922-01A,0.197100,-0.11991,0.061087,-0.476420,0.61476,-0.61644,-0.30777,-0.755140,0.42579,-0.087877,...,-0.201523,-0.109770,-0.018871,-0.024563,0.23147,-0.140680,0.192026,0.312169,-1.031938,-0.110418
TCGA-CM-5341-01A,0.102780,-0.28875,0.986980,0.228500,0.06715,-0.47900,-0.15629,-0.582330,-0.48050,-0.370740,...,-0.515830,-0.505400,-0.263510,0.149780,1.48020,-0.018243,0.056647,0.483704,-0.813626,-0.224691
TCGA-DM-A0X9-01A,-0.127710,-0.14024,0.108530,0.731420,-0.35289,-0.88487,-0.20602,-0.294110,-0.26123,0.639740,...,-0.284889,0.021421,-0.248100,-0.309530,0.34952,-0.083643,0.484082,0.459417,-0.832135,-0.353984
